# Modeling

In this notebook, the performance of different models is examined.

## Setup

Import libraries.

In [96]:
import numpy as np
import pandas as pd
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE
from sklearn.compose import make_column_transformer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from xgboost import XGBClassifier

Load datasets.

In [97]:
df_train = pd.read_csv("../data/processed/train.csv")
df_val = pd.read_csv("../data/processed/val.csv")
df_test = pd.read_csv("../data/processed/test.csv")

In [98]:
X_train = df_train.drop(columns=["claim_number", "fraud"])
y_train = df_train["fraud"]
X_val = df_val.drop(columns=["claim_number", "fraud"])
y_val = df_val["fraud"]
X_test = df_test.drop(columns=["claim_number"])

## Model Selection

`OneHotEncoder` will dummify categorical features, and numerical features will be re-scaled with `MinMaxScaler`.

In [99]:
categorical_features = X_train.columns[X_train.dtypes == object].tolist()
column_transformer = make_column_transformer(
    (OneHotEncoder(drop="first"), categorical_features),
    remainder="passthrough",
)
scaler = MinMaxScaler()

A simple function that defines the training pipeline: fit the model, predict on the validation set, print the evaluation metric.

In [100]:
def modeling(X_train, y_train, X_val, y_val, steps):
    pipeline = make_pipeline(*steps)
    pipeline.fit(X_train, y_train)
    y_val_pred = pipeline.predict_proba(X_val)[:, 1]
    metric = roc_auc_score(y_val, y_val_pred)
    if isinstance(pipeline._final_estimator, RandomizedSearchCV) or isinstance(pipeline._final_estimator, GridSearchCV):
        print(f"Best params: {pipeline._final_estimator.best_params_}")
    print(f"AUC score: {metric}")
    return pipeline

### K-Nearest Neighbor

KNN has two hyperparameters: the number of neighbors, and whether all points in each neighborhood are weighted equally or weighted by the inverse of their distance. Since the number of hyperparameters is small. A grid search is used to find the optimal hyperparameter values.

In [101]:
param_grid = {
    "n_neighbors": [5, 10, 25, 50],
    "weights": ["uniform", "distance"],
}

knn_clf = GridSearchCV(
    KNeighborsClassifier(),
    param_grid=param_grid,
    n_jobs=-1,
    cv=5,
    scoring="roc_auc",
)

knn_pipeline = modeling(X_train, y_train, X_val, y_val, [column_transformer, scaler, knn_clf])

Best params: {'n_neighbors': 50, 'weights': 'distance'}
AUC score: 0.6507841602442943


### Logistic Regression

For logistic regression, there is no hyperparameter to tune.

In [102]:
lr_clf = LogisticRegression()
lr_pipeline = modeling(X_train, y_train, X_val, y_val, [column_transformer, scaler, lr_clf])

AUC score: 0.7158620196696263


Look at the model coefficients.

In [103]:
def add_dummies(df, categorical_features):
    dummies = pd.get_dummies(df[categorical_features], drop_first=True)
    df = pd.concat([dummies, df], axis=1)
    df = df.drop(categorical_features, axis=1)
    return df.columns

feature_names = add_dummies(X_train, categorical_features)

pd.DataFrame({
    "feature_name": feature_names,
    "coefficient": lr_pipeline._final_estimator.coef_[0]
}).sort_values(by="coefficient", ascending=False).reset_index(drop=True)

,feature_name,coefficient
0,past_num_of_claims,1.744461
1,annual_income,1.651437
2,age_of_vehicle,0.973873
3,address_change_ind,0.398695
4,longitude,0.355322
5,living_status_Rent,0.128365
6,vehicle_weight,0.110477
7,vehicle_price,0.086373
8,policy_report_filed_ind,0.085690
9,channel_Phone,0.041538


### XGBoost

Since there are many hyperparameters in XGBoost, I decide to use a randomized search for hyperparameter tuning.

In [104]:
param_grid = {
    "max_depth": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "learning_rate": [0.001, 0.01, 0.1, 0.2, 0.3],
    "subsample": [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    "colsample_bytree": [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    "colsample_bylevel": [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    "min_child_weight": [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
    "gamma": [0, 0.25, 0.5, 1.0],
    "n_estimators": [10, 20, 40, 60, 80, 100, 150, 200]
}

xgb_clf = RandomizedSearchCV(
    XGBClassifier(),
    param_distributions=param_grid,
    n_iter=50,
    n_jobs=-1,
    cv=5,
    random_state=23,
    scoring="roc_auc",
)

xgb_pipeline = modeling(X_train, y_train, X_val, y_val, [column_transformer, scaler, xgb_clf])

Best params: {'subsample': 0.6, 'n_estimators': 200, 'min_child_weight': 5.0, 'max_depth': 1, 'learning_rate': 0.2, 'gamma': 0.25, 'colsample_bytree': 0.8, 'colsample_bylevel': 1.0}
AUC score: 0.7299492498801847


Although the class imbalance is not very serious in this dataset, I want to see if using SMOTE to synthesize new examples for the minority class can improve the predictive performance. However, it seems that using SMOTE only worsens the performance.

In [105]:
sampler = SMOTE(random_state=42)
xgb_pipeline_smote = modeling(X_train, y_train, X_val, y_val, [column_transformer, scaler, sampler, xgb_clf])

Best params: {'subsample': 1.0, 'n_estimators': 200, 'min_child_weight': 0.5, 'max_depth': 10, 'learning_rate': 0.1, 'gamma': 0.25, 'colsample_bytree': 0.5, 'colsample_bylevel': 0.6}
AUC score: 0.692908458782958


Save the XGBoost model (without SMOTE), since it has the best performance.

In [106]:
best_model = xgb_pipeline._final_estimator.best_estimator_
steps = [column_transformer, scaler, best_model]
pipeline = make_pipeline(*steps)
y_test_pred = pipeline.predict_proba(X_test)[:, 1]

df = pd.DataFrame({
    "claim_number": df_test["claim_number"],
    "fraud": y_test_pred
})
df.to_csv("../data/submission/submission.csv", index=False)

To examine which feature is important, I introduce a feature with random numbers. A feature can be considered as important If the importance of that feature is larger than that of the random feature.

In [107]:
X_train["random_feature"] = np.random.uniform(size=len(X_train))
xgb_clf_random_feature = XGBClassifier(**xgb_pipeline._final_estimator.best_params_)
steps = [column_transformer, scaler, xgb_clf_random_feature]
xgb_pipeline_random_feature = make_pipeline(*steps)
xgb_pipeline_random_feature = xgb_pipeline_random_feature.fit(X_train, y_train)

pd.DataFrame({
    "feature_name": list(feature_names) + ["random_feature"],
    "importance": xgb_pipeline_random_feature._final_estimator.feature_importances_
}).sort_values(by="importance", ascending=False).reset_index(drop=True)

,feature_name,importance
0,accident_site_Parking Lot,0.111512
1,high_education_ind,0.086793
2,witness_present_ind,0.073076
3,past_num_of_claims,0.049883
4,marital_status,0.049224
5,address_change_ind,0.042440
6,annual_income,0.038600
7,claim_est_payout,0.037326
8,longitude,0.035035
9,age_of_driver,0.034031
